<a href="https://colab.research.google.com/github/vkjdinesh/Reseacrh/blob/main/AI_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Go to Runtime and change the runtime to "GPU"

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install Flask==3.0.0
!pip install flask-ngrok==0.0.25
!pip install nltk==3.8.1
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.0.2-py3-none-any.whl (819 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.27.7-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.0 MB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
    Using cached distro-1.8.0-py3-none-any.whl (20 kB)
    Using cached packaging-23.2-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.42.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-_e6vm8d0/overlay/local/bin
    changing mode of /tmp/pip-build-env-_e6vm8d0/overlay/local/bin/ninja to 755
    changing mod

In [ ]:
!mkdir /root/.ngrok2/
!echo -e "version: '2'\nregion: jp\nauthtoken: <auth_token>" > ~/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2/’: File exists


In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
#float value you can update it to 32 if there is any error popup
pipe = pipe.to("cuda")

import os
import matplotlib.pyplot as plt

safety_checker/pytorch_model.fp16.safetensors not found


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
!pip install Flask-Cors==4.0.0
!pip install Pillow==10.1.0

In [ ]:
import os
import io
from PIL import Image

from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

import nltk

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

html = """
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <style>
      :root {
        --clr-green: #a3e635;
        --clr-dark-blue: #172554;
        --clr-white: #f8fafc;
      }

      * {
        padding: 0;
        margin: 0;
        font-family: "Franklin Gothic Medium", "Arial Narrow", Arial, sans-serif;
        border-radius: 0px;
      }

      body {
        background-color: var(--clr-dark-blue);
      }

      .section {
        height: 100%;
      }

      .prompt {
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        gap: 50px;
      }

      .btn {
        width: 200px;
        height: 50px;
        border: none;
      }

      .prompt__title {
        color: var(--clr-green);
        font-size: 100px;
      }

      .prompt__textfield {
        width: 600px;
        height: 70px;
        padding: 10px;
        text-align: center;
        background-color: var(--clr-green);
        border: none;
      }

      .story {
        display: flex;
        justify-content: center;
        align-items: center;
        gap: 70px;
        color: var(--clr-white);
        margin: auto 70px;
      }

      .story__story {
        width: 600px;
        display: flex;
        flex-direction: column;
        justify-content: start;
        gap: 20px;
      }

      .story__story-title {
        text-align: center;
        font-size: 60px;
      }

      .story__story-line {
        width: 100%;
        height: 5px;
        background-color: var(--clr-green);
      }

      .sentence {
        font-size: 20px;
      }

      .sentence:hover {
        background-color: var(--clr-green);
        cursor: pointer;
      }

      .story__image {
        width: 600px;
        display: flex;
        flex-direction: column;
        justify-content: start;
        align-items: center;
        gap: 20px;
      }

      .story__image-title {
        font-size: 40px;
      }

      .box {
        height: 50px;
        width: 100%;
        color: black;
        text-align: center;
        background-color: var(--clr-green);
      }

      .story__image-img {
        width: 100%;
      }

      .mcq {
        display: flex;
        flex-direction: column;
        align-items: center;
        gap: 20px;
      }

      .mcq__title {
        font-size: 40px;
        color: white;
      }

      .mcq__line {
        width: 50%;
        height: 5px;
        background-color: var(--clr-green);
      }

      .mcq__text {
        font-size: 20px;
        width: 50%;
        color: white;
      }
    </style>
    <title>AI Web App</title>
  </head>
  <body>
    <div class="section prompt">
      <h1 class="prompt__title">Enter the Text prompt.</h1>
      <form>
        <input
          class="prompt__textfield"
          type="text"
          name="prompt"
          placeholder="Two dogs playing together on a beach"
        />
      </form>
      <button class="btn" onclick="submit()">Enter</button>
    </div>

    <div class="section story">
      <div class="story__story">
        <h1 class="story__story-title">Short Story</h1>
        <div class="story__story-line"></div>
        <div class="story__story-text"></div>
      </div>
      <div class="story__image">
        <h1 class="story__image-title">Image</h1>
        <div>
          <div class="box">
            Max and Bella, two inseparable dogs, were playing joyfully on a
            sun-kissed beach.
          </div>
          <img
            class="story__image-img"
            src="https://png.pngtree.com/thumb_back/fw800/background/20220111/pngtree-fighting-dogs-on-the-beach-two-angry-sociability-photo-image_26163740.jpg"
            alt="generated picture"
          />
        </div>
        <button class="btn" onclick="showMcq()">MCQ</button>
      </div>
    </div>

    <div class="section mcq">
      <h1 class="mcq__title">MCQs</h1>
      <div class="mcq__line"></div>
      <div class="mcq__text"></div>
    </div>

    <script src="https://code.jquery.com/jquery-3.7.1.min.js" integrity="sha256-/JqT3SQfawRcv/BIHPThkBvs0OEvtFFmqPF/lYI/Cxo=" crossorigin="anonymous"></script>
    <script>
        function submit() {
            console.log("hit js")
            let prompt = document.querySelector(".prompt__textfield").value;
            console.log(prompt);
            $.ajax({
                url: '/handle_prompt',
                type: 'POST',
                data: {prompt: prompt},
                success: function(response) {
                    story = response.result[0];
                    alert(story);
                    let div = document.querySelector(".story__story-text");
                    div.textContent = "";
                    story.forEach((sentence, x) => {
                        let p = document.createElement("p");
                        let br = document.createElement("br");

                        p.textContent = sentence;
                        p.classList.add("sentence");
                        p.onclick = getImage;

                        div.appendChild(p);
                        div.appendChild(br);
                    })

                    let storySection = document.querySelector(".story");
                    storySection.scrollIntoView({ behavior: "smooth" });
                    console.log("done creating sentences");

                    mcq = response.result[1];
                    div = document.querySelector(".mcq__text");
                    mcq.forEach((mcqq, x) => {
                        let p = document.createElement("p");

                        p.textContent = mcqq;

                        div.appendChild(p);
                    })
                },
                error: function(error) {
                    console.log("error from ajax: " + JSON.stringify(error));
                }
            });
        }

        function getImage(event) {
            let ele = event.target
            console.log(ele.innerHTML)
            let img = document.querySelector(".story__image-img");
            img.src = "/handle_image/" + ele.innerHTML;

            let caption = document.querySelector(".box");
            caption.innerHTML = ele.innerHTML;

            console.log("changed image")
          }

        function showMcq() {
          let mcqSection = document.querySelector(".mcq");
          mcqSection.scrollIntoView({ behavior: "smooth" });
        }
    </script>
  </body>
</html>
"""

@app.route('/handle_prompt', methods=['POST'])
def handle_prompt():
    print('hit py')
    prompt = request.form.get('prompt')
    prompt_template = f'''
    USER: {prompt + "1.give short story with 200 words. 2. generate 5 MCQs with options and answer"}

    ASSISTANT:
    '''
    response = lcpp_llm(prompt=prompt_template, max_tokens=800, temperature=0.5, top_p=0.95,
                        repeat_penalty=1.2, top_k=150,
                        echo=True)

    res = response["choices"][0]["text"]
    res = res.split(prompt_template)[1] # getting only the AI's response
    print(res)

    try:
        story, mcq = res.split("1.")
    except ValueError:
        story, mcq = res.split("1)")

    if story.count(":") == 2:
        story = story.split(":")[1].strip()
    elif story.count(":") == 3:
        story = story.split(":")[2].strip()
    elif story.count(":") == 5:
        story = story.split(":")[5].strip()

    mcq = "1." + mcq

    nltk.download('punkt')
    story = nltk.sent_tokenize(story)
    mcq = nltk.sent_tokenize(mcq)
    story = story[:-1] # exclude the last sentence

    print(story, mcq)
    return jsonify(result=[story, mcq])

@app.route('/handle_image/<prompt>', methods=['GET'])
def handle_image(prompt):
    print('hit py')
    # prompt = request.form.get('prompt')
    image = pipe(prompt + " Japanese Anime").images[0]
    image.save("output_image.jpeg")

    pil_img = Image.open("output_image.jpeg")
    image_io = io.BytesIO()
    pil_img.save(image_io, 'JPEG', quality=70)
    image_io.seek(0)

    os.remove("output_image.jpeg")

    return send_file(image_io, mimetype='image/jpeg')

@app.route("/")
def home():
    return html

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c734-35-233-207-121.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Dec/2023 03:46:35] "POST /handle_prompt HTTP/1.1" 200 -


['"The sun was shining brightly as John and his six-year-old son, Max, set out on their camping trip.', 'They packed all of their gear into the car and hit the road, excited for their adventure.', 'As they drove deeper into the mountains, the air grew cooler and the trees became taller.', 'Finally, they arrived at their campsite - a secluded spot near a babbling brook.', 'John and Max set up their tent and started a fire to cook dinner.', 'That night, under the stars, they roasted marshmallows and told ghost stories."'] ['1.', 'What was the weather like when John and Max left for their trip?', 'A) Sunny\n        B) Cloudy\n        C) Rainy\n        D) Snowy\n    2.', 'Where did John and Max go camping?', 'A) The beach\n        B) The mountains\n        C) The forest\n        D) The city\n    3.', 'What did John and Max set up at their campsite?', 'A) A tent\n        B) A hammock\n        C) A cabin\n        D) None of the above\n    4.', 'What did they use to cook dinner?', 'A) A stove

...

  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [01/Dec/2023 03:51:03] "GET /handle_image/A%20father%20and%20son%20went%20on%20a%20trip%20together%20to%20explore%20new%20places. HTTP/1.1" 200 -


hit py


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [01/Dec/2023 03:51:22] "GET /handle_image/They%20stopped%20at%20a%20local%20café%20for%20lunch%20and%20tried%20some%20delicious%20food. HTTP/1.1" 200 -


In [ ]:
# Get the prompt from the user
user_prompt = input("Please enter your prompt: ")

# Now you can use the 'user_prompt' in your code
prompt_template = f'''
USER: {user_prompt + "1.give short story with 200 words. 2. generate 5 MCQs with options and answer"}

ASSISTANT:
'''
response = lcpp_llm(prompt=prompt_template, max_tokens=800, temperature=0.5, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=True)



Please enter your prompt: a father and son go for a trip


Llama.generate: prefix-match hit


In [ ]:
res = response["choices"][0]["text"]
result = res.split('Now, here are five multiple-choice questions based on the story:')

In [ ]:
print(result[0])


USER: a father and son go for a trip1.give short story with 200 words. 2. generate 5 MCQs with options and answer

ASSISTANT:

Sure, I'd be happy to help! Here is a short story about a father and son going on a trip:

Once upon a time, a father and his young son set out on an adventurous road trip. They packed their bags, loaded up the car, and hit the open road. As they drove through the rolling hills and picturesque towns, they talked about everything from sports to school to their favorite movies.

As they reached their destination, a beautiful lake nestled in the mountains, they couldn't wait to explore. They spent their days fishing, hiking, and swimming in the crystal-clear waters. At night, they sat around a campfire, roasting marshmallows and telling stories under the stars.

The trip was filled with laughter, adventure, and quality time together. The father and son bonded over their shared experiences and created memories that would last a lifetime.




In [ ]:
print(result[1])



1. Where did the father and son go on their road trip?
a) Across the country
b) To a nearby city
c) On a hike in the mountains
d) To an amusement park

Answer: a) Across the country

2. What was the purpose of the father and son's road trip?
a) To visit famous landmarks
b) To go on a hiking adventure
c) To explore new places
d) To spend time together as a family

Answer: c) To explore new places

3. What did the father and son do at each of the theme parks they visited?
a) They rode roller coasters all day long
b) They explored the park's history and culture
c) They went on a hike through the surrounding woods
d) They spent time together, bonding over their shared love of adventure

Answer: d) They spent time together, bonding over their shared love of adventure

4. How did the father and son feel about each other during the road trip?
a) They were bored with each other's company
b) They were excited to be exploring new places together
c) They were frustrated with each other's differ